In [1]:
def solve_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs):
    """
    Solve a multi-state multi-knapsack problem using a classical backtracking algorithm.
    
    Args:
        profit_values (2D list of float): Profit values for the possible states for each item.
        cost_values (3D list of float): Cost values for the possible states for each item in each knapsack.
                                        cost_values[s][i][k] means the cost of item i in state s for knapsack k.
        max_total_costs (list of float): Maximum allowable total cost for each knapsack.
        
    Returns:
        list: The optimal selection of states for each item.
        list: The allocation of each item to knapsacks.
        float: The maximum total profit.
        
    # len(profit_values) == len(cost_values)
    # len(profit_values[0]) == len(cost_values[0])
    # len(max_total_costs) == number of knapsacks
    """
    n = len(profit_values[0])  # Number of items
    num_knapsacks = len(max_total_costs)
    num_states = len(profit_values)
    
    def backtrack(index, current_costs, current_profit, current_solution, current_knapsack_allocation):
        """
        Recursive backtracking function to search for the optimal solution.
        
        Args:
            index (int): Current item index being processed.
            current_costs (list of float): Current total cost used in each knapsack.
            current_profit (float): Current total profit of the solution path.
            current_solution (list of int): Current list of selected states for items.
            current_knapsack_allocation (list of int): Current list of knapsack assignments for items.
        
        Returns:
            list: The best solution (state selection) for the current path.
            list: The best knapsack allocation for the current path.
            float: The total profit of the best solution.
        """
        if index == n:  # Base case: all items are processed
            return current_solution, current_knapsack_allocation, current_profit
        
        best_solution, best_allocation, best_profit = None, None, -float('inf')
        
        for state in range(num_states):  # Possible states for each item
            for knapsack in range(num_knapsacks):  # Possible knapsacks for each item
                new_costs = current_costs[:]
                new_costs[knapsack] += cost_values[state][index][knapsack]
                
                if all(new_costs[k] <= max_total_costs[k] for k in range(num_knapsacks)):  # Prune paths exceeding the cost limits
                    solution, allocation, profit = backtrack(
                        index + 1, 
                        new_costs, 
                        current_profit + profit_values[state][index], 
                        current_solution + [state], 
                        current_knapsack_allocation + [knapsack]
                    )
                    
                    if profit > best_profit:
                        best_solution, best_allocation, best_profit = solution, allocation, profit
        
        return best_solution, best_allocation, best_profit

    # Start the backtracking process from index 0
    initial_costs = [0] * num_knapsacks
    solution, knapsack_allocation, max_profit = backtrack(0, initial_costs, 0, [], [])
    return solution, knapsack_allocation, max_profit



In [2]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram

def formulate_qp_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs):
    """
    Formulate the multi-state multi-knapsack problem as a quadratic program using Qiskit's QuadraticProgram.
    
    Args:
        profit_values (2D list of float): Profit values for the possible states for each item.
        cost_values (3D list of float): Cost values for the possible states for each item in each knapsack.
                                        cost_values[s][i][k] means the cost of item i in state s for knapsack k.
        max_total_costs (list of float): Maximum allowable total cost for each knapsack.
    
    Returns:
        QuadraticProgram: The formulated quadratic program for QAOA-based solution.
    """
    n = len(profit_values[0])  # Number of items
    num_states = len(profit_values)  # Number of possible states per item
    num_knapsacks = len(max_total_costs)  # Number of knapsacks
    
    qp = QuadraticProgram("Multi-State Multi-Knapsack Problem")

    # Step 1: Define binary variables for each item's possible state and knapsack allocation
    for i in range(n):
        for s in range(num_states):
            for k in range(num_knapsacks):
                qp.binary_var(name=f"x_{s}_{i}_{k}") 

    # Step 2: Objective function: maximize profits
    linear_terms = {}
    for i in range(n):
        for s in range(num_states):
            for k in range(num_knapsacks):
                linear_terms[f"x_{s}_{i}_{k}"] = profit_values[s][i]  # Profit of item i in state s

    qp.maximize(linear=linear_terms)

    # Step 3: Constraint 1: Each item must be assigned to exactly one state in one knapsack
    for i in range(n):
        qp.linear_constraint(
            linear={f"x_{s}_{i}_{k}": 1 for s in range(num_states) for k in range(num_knapsacks)},
            sense="==",
            rhs=1,
            name=f"state_and_knapsack_assignment_constraint_{i}",
        )

    # Step 4: Constraint 2: Total cost for each knapsack must not exceed the maximum allowable cost
    for k in range(num_knapsacks):
        cost_terms = {}
        for i in range(n):
            for s in range(num_states):
                cost_terms[f"x_{s}_{i}_{k}"] = cost_values[s][i][k]  # Cost of item i in state s for knapsack k
        qp.linear_constraint(
            linear=cost_terms,
            sense="<=",
            rhs=max_total_costs[k],
            name=f"total_cost_constraint_knapsack_{k}",
        )

    return qp


def solve_qp_with_qiskit(profit_values, cost_values, max_total_costs):
    """
    Solve the quadratic program using Qiskit's optimization solvers.

    Args:
        profit_values (2D list of float): Profit values for each item in each state.
        cost_values (3D list of float): Cost values for each item in each state for each knapsack.
        max_total_costs (list of float): Maximum allowable total cost for each knapsack.

    Returns:
        dict: Solution details including selected items and total revenue.
    """
    qp = formulate_qp_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs)

    print(qp.prettyprint())
    
    qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
    optimizer = MinimumEigenOptimizer(qaoa)
    result = optimizer.solve(qp)
    solution = result.x

    total_profit = 0
    assignments = []

    
    n = len(profit_values[0])  # Number of items
    num_knapsacks = len(max_total_costs)
    num_states = len(profit_values)

    for i in range(n):  # For each item
        for s in range(num_states):  # For each state
            for k in range(num_knapsacks):  # For each knapsack
                index = (num_knapsacks * num_states * i) + (num_knapsacks * s) + k
                if solution[index] == 1:
                    total_profit += profit_values[s][i]
                    assignments.append((i, s, k))  # (item, state, knapsack)

    return {
        "assignments": assignments,
        "total_profit": total_profit,
        "ansatz": qaoa.ansatz
    }



In [ ]:
profit_values = [
    [5, 10],   # Profits if item i is in state 0
    [8, 15],   # Profits if item i is in state 1
    [12, 20]   # Profits if item i is in state 2
]

cost_values = [
    [  # Costs for items if in state 0
        [10, 12], [15, 18], 
    ],
    [  # Costs for items if in state 1
        [11, 13], [16, 19], 
    ],
    [  # Costs for items if in state 2
        [14, 17], [20, 22], 
    ]
]

max_total_costs = [20, 25]  # Max cost for knapsack 0 and 1

# Solve the problem
solution, knapsack_allocation, max_profit = solve_multi_state_multi_knapsack(profit_values, cost_values, max_total_costs)

print(f"Optimal state selection: {solution}")
print(f"Knapsack allocation: {knapsack_allocation}")
print(f"Maximum profit: {max_profit}")

result = solve_qp_with_qiskit(profit_values, cost_values, max_total_costs)
print("Assignments (item, state, knapsack):", result['assignments'])
print("Total profit:", result['total_profit'])


Optimal state selection: [2, 2]
Knapsack allocation: [0, 1]
Maximum profit: 32
Problem name: Multi-State Multi-Knapsack Problem

Maximize
  5*x_0_0_0 + 5*x_0_0_1 + 10*x_0_1_0 + 10*x_0_1_1 + 8*x_1_0_0 + 8*x_1_0_1
  + 15*x_1_1_0 + 15*x_1_1_1 + 12*x_2_0_0 + 12*x_2_0_1 + 20*x_2_1_0 + 20*x_2_1_1

Subject to
  Linear constraints (4)
    x_0_0_0 + x_0_0_1 + x_1_0_0 + x_1_0_1 + x_2_0_0 + x_2_0_1
    == 1  'state_and_knapsack_assignment_constraint_0'
    x_0_1_0 + x_0_1_1 + x_1_1_0 + x_1_1_1 + x_2_1_0 + x_2_1_1
    == 1  'state_and_knapsack_assignment_constraint_1'
    10*x_0_0_0 + 15*x_0_1_0 + 11*x_1_0_0 + 16*x_1_1_0 + 14*x_2_0_0 + 20*x_2_1_0
    <= 20  'total_cost_constraint_knapsack_0'
    12*x_0_0_1 + 18*x_0_1_1 + 13*x_1_0_1 + 19*x_1_1_1 + 17*x_2_0_1 + 22*x_2_1_1
    <= 25  'total_cost_constraint_knapsack_1'

  Binary variables (12)
    x_0_0_0 x_0_0_1 x_1_0_0 x_1_0_1 x_2_0_0 x_2_0_1 x_0_1_0 x_0_1_1 x_1_1_0
    x_1_1_1 x_2_1_0 x_2_1_1



C:\Users\forca\AppData\Local\Temp\ipykernel_16028\1339769304.py:82: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA())
